In [1]:
!pip install wheel==0.37.1 setuptools==59.6.0
!pip install gym[atari,accept-rom-license]==0.21.0 tensorflow-probability==0.19.0 tqdm==4.64.1

     |████████████████████████████████| 952 kB 6.5 MB/s eta 0:00:01
  Attempting uninstall: wheel
    Found existing installation: wheel 0.34.2
    Uninstalling wheel-0.34.2:
      Successfully uninstalled wheel-0.34.2
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.1
    Uninstalling setuptools-65.5.1:
      Successfully uninstalled setuptools-65.5.1
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 1.5 MB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 6.7 MB 6.9 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 6.9 MB/s eta 0:00:01
     |████████████████████████████████| 152 kB 6.7 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 434 kB 7.5 MB/s eta 0:00:01
  Installing build dependencies ... done


In [2]:
from math import ceil
from dataclasses import dataclass
from typing import List

import gym
import numpy as np
from tqdm import tqdm

import tensorflow as tf
import tensorflow_probability as tfp

from tensorflow import keras
from keras import Model
from keras.layers import \
    Input, Dense, Reshape, Lambda, Flatten, \
    Conv2D, Conv2DTranspose, Dropout, Concatenate
from keras.optimizers import Adam
from keras.metrics import Mean

tfpl = tfp.layers
tfd = tfp.distributions
KLDivergenceRegularizer = tfp.layers.KLDivergenceRegularizer
MultivariateNormalTriL = tfp.layers.MultivariateNormalTriL
IndependentBernoulli = tfp.layers.IndependentBernoulli
IndependentNormal = tfp.layers.IndependentNormal
Independent = tfp.distributions.Independent
Bernoulli = tfp.distributions.Bernoulli
Normal = tfp.distributions.Normal
MultivariateNormalDiag = tfp.distributions.MultivariateNormalDiag

In [3]:
print(tf.__version__)
print(tf.test.gpu_device_name())

2.11.0
/device:GPU:0


In [4]:
@dataclass
class DreamerSettings:
    action_dims: List[int]
    obs_dims: List[int]
    repr_dims: List[int]
    hidden_dims: List[int]
    enc_dims: List[int]
    dropout_rate: float = 0.2

    @property
    def repr_dims_flat(self) -> int:
        return self.repr_dims[0] * self.repr_dims[1]

    @property
    def repr_out_dims_flat(self) -> int:
        return self.repr_dims[0] * self.repr_dims[1] + self.hidden_dims[0]

In [5]:
def sample_obs(env: gym.Env, num_obs: int) -> tf.data.Dataset:
    all_obs = []
    done = False
    obs = env.reset()
    all_obs.append(obs)
    while len(all_obs) < num_obs:
        action = env.action_space.sample()
        obs, reward, done, _ = env.step(action)
        all_obs.append(obs)
        if done and len(all_obs) < num_obs:
            obs = env.reset()
            all_obs.append(obs)
    return tf.data.Dataset.from_tensor_slices(tensors=(np.array(all_obs)))

In [6]:
#env = gym.make("ALE/Pacman-v5")
#settings = DreamerSettings([1], [64, 64, 3], [32, 32], [512], [16])

settings = DreamerSettings([1], [64, 64, 1], [32, 32], [512], [16])

In [16]:
#batch_size = 64
#timesteps = 10_000
#dataset = sample_obs(env, timesteps)
#dataset = dataset.map(lambda obs: tf.image.resize(obs, settings.obs_dims[:2]))
#dataset = dataset.map(lambda obs: (obs / 255.0, obs / 255.0))
#num_eval_batches = ceil((timesteps // batch_size) * 0.1)
#dataset = dataset.batch(batch_size)
#train_dataset = dataset.skip(num_eval_batches)
#eval_dataset = dataset.take(num_eval_batches)
#train_dataset = train_dataset.shuffle(100)

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
train_imgs = tf.data.Dataset.from_tensor_slices(tensors=(x_train.astype(np.float32)))
test_imgs = tf.data.Dataset.from_tensor_slices(tensors=(x_test.astype(np.float32)))

train_imgs = train_imgs.map(lambda obs: tf.reshape(obs, (28, 28, 1)))
test_imgs = test_imgs.map(lambda obs: tf.reshape(obs, (28, 28, 1)))
train_imgs = train_imgs.map(lambda obs: tf.image.resize(obs, settings.obs_dims[:2]))
test_imgs = test_imgs.map(lambda obs: tf.image.resize(obs, settings.obs_dims[:2]))

train_dataset = train_imgs.map(lambda obs: (obs / 255.0, obs / 255.0))
eval_dataset = test_imgs.map(lambda obs: (obs / 255.0, obs / 255.0))

batch_size = 64
train_dataset = train_dataset.batch(batch_size)
eval_dataset = eval_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(100)

print(next(iter(train_dataset))[0].shape)


KeyboardInterrupt



In [14]:
def create_state_encoder_model(settings: DreamerSettings) -> Model:
    enc_units = settings.enc_dims[0] * 2
    model_in = Input(settings.obs_dims, name="obs")
    norm_img = Lambda(lambda x: x * 2.0 - 1.0)
    cnn_1 = Conv2D(64, (3, 3), strides=(2, 2), padding="same", activation="elu")
    cnn_2 = Conv2D(64, (3, 3), strides=(2, 2), padding="same", activation="elu")
    cnn_3 = Conv2D(64, (3, 3), strides=(2, 2), padding="same", activation="elu")
    cnn_4 = Conv2D(8, (3, 3), padding="same", activation="elu")
    drop_1 = Dropout(rate=settings.dropout_rate)
    drop_2 = Dropout(rate=settings.dropout_rate)
    drop_3 = Dropout(rate=settings.dropout_rate)
    drop_4 = Dropout(rate=settings.dropout_rate)
    flatten = Flatten()
    dense_out = Dense(enc_units, activation="linear", name="enc_dense")

    img_in = norm_img(model_in)
    prep_model_convs = drop_4(cnn_4(drop_3(cnn_3(drop_2(cnn_2(drop_1(cnn_1(img_in))))))))
    model_out = dense_out(flatten(prep_model_convs))
    return Model(inputs=model_in, outputs=model_out, name="encoder_model")


def create_representation_model(settings: DreamerSettings) -> Model:
    enc_dims = settings.enc_dims[0]
    model_in = Input((enc_dims * 2))

    prior = tfd.Independent(tfd.Normal(loc=tf.zeros(enc_dims), scale=1))
    split = Lambda(lambda x: (x[:, :enc_dims], x[:, enc_dims:]))
    posterior = Lambda(lambda x: x[0] + x[2].sample() * tf.math.exp(x[1] / 2.0))

    mu, sigma = split(model_in)
    model_out = posterior((mu, sigma, prior))
    return Model(inputs=[model_in], outputs=[model_out, (mu, sigma)])


def create_state_decoder_model(settings: DreamerSettings) -> Model:
    image_channels = settings.obs_dims[-1]
    upscale_source_dims = (settings.obs_dims[0] // 8 * settings.obs_dims[1] // 8) * 8

    model_in = Input(settings.enc_dims[0], name="repr_out")
    dense_in = Dense(upscale_source_dims, activation="linear", name="dec_in")
    reshape_in = Reshape((settings.obs_dims[0] // 8, settings.obs_dims[1] // 8, -1))
    cnn_1 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same", activation="elu")
    cnn_2 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same", activation="elu")
    cnn_3 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same", activation="elu")
    cnn_4 = Conv2D(64, (3, 3), padding="same", activation="elu")
    cnn_5 = Conv2D(image_channels, (1, 1), padding="same", activation="linear")
    drop_1 = Dropout(rate=settings.dropout_rate)
    drop_2 = Dropout(rate=settings.dropout_rate)
    drop_3 = Dropout(rate=settings.dropout_rate)

    prep_in = reshape_in(dense_in(model_in))
    model_out = cnn_5(cnn_4(drop_3(cnn_3(drop_2(cnn_2(drop_1(cnn_1(prep_in))))))))
    return Model(inputs=model_in, outputs=model_out, name="decoder_model")


def compose_model(settings: DreamerSettings) -> Model:
    encoder = create_state_encoder_model(settings)
    latent_model = create_representation_model(settings)
    decoder = create_state_decoder_model(settings)
    flatten = Flatten()
    concat = Concatenate()

    model_in = Input(settings.obs_dims)
    enc_out = encoder(model_in)
    z, (mu, log_sigma) = latent_model(enc_out)
    reconst_img = decoder(z)

    return Model(inputs=[model_in], outputs=[reconst_img, mu, log_sigma])

In [15]:
model = compose_model(settings)
model.build([None] + settings.obs_dims)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 64, 64, 1)]       0         
                                                                 
 encoder_model (Functional)  (None, 32)                95528     
                                                                 
 model_2 (Functional)        [(None, 16),              0         
                              ((None, 16),                       
                              (None, 16))]                       
                                                                 
 decoder_model (Functional)  (None, 64, 64, 1)         124225    
                                                                 
Total params: 219,753
Trainable params: 219,753
Non-trainable params: 0
_________________________________________________________________


In [12]:
optimizer = Adam()
summary_writer = tf.summary.create_file_writer("logs/vae")


@tf.function
def train_step(batch):
    x, y_true = batch

    with tf.GradientTape() as tape:
        reconst_image, mu, logsigma = model(x)
        reconst_loss = tf.reduce_mean((reconst_image - y_true)**2)
        elbo_loss = -tf.reduce_mean(1.0 + logsigma - mu**2 - tf.math.exp(logsigma))
        loss = reconst_loss + 0.1 * elbo_loss

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return reconst_loss, elbo_loss


@tf.function
def eval_step(batch):
    x, y_true = batch

    reconst_image, mu, logsigma = model(x)
    reconst_loss = tf.reduce_mean((reconst_image - y_true)**2)
    elbo_loss = -tf.reduce_mean(1.0 + logsigma - mu**2 - tf.math.exp(logsigma))

    return reconst_loss, elbo_loss


def log_losses(step, reconst_loss, elbo_loss, mode: str):
    with summary_writer.as_default():
        tf.summary.scalar(f"{mode}/reconst_loss", reconst_loss, step=step)
        tf.summary.scalar(f"{mode}/elbo_loss", elbo_loss, step=step)
        tf.summary.scalar(f"{mode}/total_loss", reconst_loss + elbo_loss, step=step)


def train():
    epochs = 500
    mean_reconst_loss = Mean()
    mean_elbo_loss = Mean()

    for ep in tqdm(range(epochs)):
        for batch in train_dataset:
            reconst_loss, elbo_loss = train_step(batch)
            mean_reconst_loss(reconst_loss)
            mean_elbo_loss(elbo_loss)

        reconst_loss = mean_reconst_loss.result()
        elbo_loss = mean_elbo_loss.result()
        log_losses(ep, reconst_loss, elbo_loss, "train")
        mean_reconst_loss.reset_states()
        mean_elbo_loss.reset_states()

        for batch in eval_dataset:
            reconst_loss, elbo_loss = eval_step(batch)
            mean_reconst_loss(reconst_loss)
            mean_elbo_loss(elbo_loss)

        reconst_loss = mean_reconst_loss.result()
        elbo_loss = mean_elbo_loss.result()
        log_losses(ep, reconst_loss, elbo_loss, "eval")
        mean_reconst_loss.reset_states()
        mean_elbo_loss.reset_states()


train()

  0%|          | 0/500 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

def display_imgs(labels, preds):
    plt.ioff()
    n = labels.shape[0]
    fig, axs = plt.subplots(n, 3, figsize=(128, 128))
    for i in range(n):
        diff = preds[i] - labels[i]
        min_val, max_val = np.min(diff), np.max(diff)
        diff = (diff - min_val) / (max_val - min_val) if max_val - min_val != 0 else diff

        axs[i, 0].imshow(labels[i], interpolation='nearest', cmap='gray')
        axs[i, 1].imshow(preds[i], interpolation='nearest', cmap='gray')
        axs[i, 2].imshow(diff, interpolation='nearest', cmap='gray')
        axs[i, 0].axis('off')
        axs[i, 1].axis('off')
        axs[i, 2].axis('off')
        axs[i, 0].set_xticks([])
        axs[i, 0].set_yticks([])
        axs[i, 1].set_xticks([])
        axs[i, 1].set_yticks([])
        axs[i, 2].set_xticks([])
        axs[i, 2].set_yticks([])

    fig.set_size_inches(10, n*5)
    plt.subplots_adjust(wspace=0, hspace=0)

    plt.show()
    plt.close()

In [ ]:
num_rows = 20
img_dataset = eval_dataset.unbatch().shuffle(100).batch(num_rows)
img_in, _ = iter(img_dataset).next()
img_out, _, __ = model(img_in)
img_in = img_in.numpy()[:num_rows]
img_out = np.clip(img_out.numpy()[:num_rows], 0, 1)

display_imgs(img_in, img_out)